In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sc
from cvxopt import solvers,matrix 

In [2]:
import os
os.listdir()

['state_of_the_art_soft_margin.ipynb',
 '.DS_Store',
 'state_of_the_art_hard_margin.ipynb',
 '.ipynb_checkpoints',
 'spamdata.mat',
 '771A_lec8_slides.pdf',
 'ACFrOgBA8JRQCnWZNF2y9MGVmmmF98EOE4PqZKLg9TZ1A0CvsQWvcuXC-3CL8jVSyp9rk6Fu0E3ujkwkgt8up0YOSIBYZFstEDQuFvTitC5bOK9Ih14Z5qbMnuOS3og=.pdf']

In [3]:
data = sc.loadmat('spamdata.mat')

In [4]:
list = ['xtrain', 'ytrain', 'xtest', 'ytest']

In [5]:
xtest, xtrain, ytest, ytrain = data['xtrain'], data['xtest'], data['ytrain'], data['ytest']

In [6]:
xtrain.shape,ytrain.shape,xtest.shape, ytest.shape

((2000, 57), (2000, 1), (500, 57), (500, 1))

In [7]:
xtrain.shape

(2000, 57)

Our dual problem is expressed as:
$\max_{\alpha} \sum_i^m \alpha_i - \frac{1}{2} \sum_{i,j}^m y^{(i)}y^{(j)} \alpha_i \alpha_j <x^{(i)} x^{(j)}>$

Let H be a matrix such that $H_{i,j} = y^{(i)}y^{(j)} <x^{(i)} x^{(j)}>$ , then the optimization becomes:

$\begin{aligned}
    & \max_{\alpha} \sum_i^m \alpha_i  - \frac{1}{2}  \alpha^T \mathbf{H}  \alpha
    \\
     s.t. & \ \alpha_i \geq 0 
    \\
    &  \ \sum_i^m \alpha_i y^{(i)} = 0  
\end{aligned}$

Equation to be solved using Solver
$\begin{aligned}
    & \min \frac{1}{2} x^TPx + q^Tx
    \\
     s.t. \ & \ Gx \leq h 
    \\
    & \ Ax = b
\end{aligned}$

In [8]:
C = 10
m,n = xtrain.shape
ytrain = ytrain.reshape(-1,1) * 1.
X_dash = ytrain * xtrain
H = np.dot(X_dash , X_dash.T) * 1.

#Converting into cvxopt format
P = matrix(H)
q = matrix(-np.ones((m, 1)))
G = matrix(np.vstack((np.eye(m)*-1,np.eye(m))))
h = matrix(np.hstack((np.zeros(m), np.ones(m) * C)))
A = matrix(ytrain.reshape(1, -1))
b = matrix(np.zeros(1))


sol = solvers.qp(P, q, G, h, A, b)
alphas = np.array(sol['x'])

     pcost       dcost       gap    pres   dres
 0: -4.3802e+03 -2.7356e+05  1e+06  2e+00  7e-08
 1: -3.4922e+03 -1.6165e+05  3e+05  3e-01  5e-08
 2: -2.6952e+03 -8.5366e+04  1e+05  1e-01  3e-08
 3: -2.4588e+03 -6.5528e+04  1e+05  1e-01  3e-08
 4: -1.8533e+03 -4.9672e+04  7e+04  5e-02  2e-08
 5: -1.5073e+03 -3.8817e+04  5e+04  2e-02  2e-08
 6: -1.7294e+03 -2.0629e+04  2e+04  1e-02  2e-08
 7: -1.7137e+03 -1.8927e+04  2e+04  7e-03  2e-08
 8: -1.8393e+03 -1.3549e+04  1e+04  3e-03  2e-08
 9: -2.1759e+03 -8.9656e+03  7e+03  1e-03  2e-08
10: -2.3909e+03 -6.9357e+03  5e+03  4e-04  2e-08
11: -2.6225e+03 -5.4620e+03  3e+03  2e-04  2e-08
12: -2.7778e+03 -4.7631e+03  2e+03  1e-04  2e-08
13: -2.8637e+03 -4.3768e+03  2e+03  7e-05  2e-08
14: -2.9727e+03 -3.8977e+03  9e+02  2e-05  2e-08
15: -3.0829e+03 -3.5961e+03  5e+02  5e-06  3e-08
16: -3.1316e+03 -3.4884e+03  4e+02  3e-06  3e-08
17: -3.1811e+03 -3.3791e+03  2e+02  9e-07  4e-08
18: -3.2086e+03 -3.3300e+03  1e+02  5e-07  2e-08
19: -3.2270e+03 -3.29

In [9]:
alphas.shape

(2000, 1)

In [10]:
w = ((ytrain * alphas).T @ xtrain).reshape(-1,1)

S = (alphas > 1e-4).flatten()

b = ytrain[S] - np.dot(xtrain[S], w)


print('Alphas = ',alphas[alphas > 1e-4])
print('w = ', w.flatten())
print('b = ', b[0])

Alphas =  [9.99999971e+00 6.50828449e+00 9.99999988e+00 9.99999997e+00
 3.69893643e+00 2.93896049e-01 9.99999860e+00 9.99999990e+00
 9.99999990e+00 9.99999973e+00 1.46820663e+00 2.97264120e+00
 9.99999955e+00 9.99999966e+00 9.99994855e+00 9.99999990e+00
 9.99999990e+00 4.09505205e+00 9.99999988e+00 9.99999933e+00
 9.99999978e+00 9.99998415e+00 9.99999929e+00 9.99999995e+00
 9.99999922e+00 3.52022730e+00 2.93901117e-01 9.99999953e+00
 9.99999988e+00 9.99999989e+00 8.15618924e+00 9.99999957e+00
 9.99999973e+00 9.99999982e+00 9.99999919e+00 9.99999627e+00
 9.99999993e+00 9.99999996e+00 9.99999989e+00 9.99999993e+00
 9.99999985e+00 9.99999974e+00 9.99999961e+00 9.99999964e+00
 9.99999986e+00 9.99999894e+00 9.99999995e+00 9.99999966e+00
 9.99999954e+00 9.99999994e+00 9.99999967e+00 9.99999961e+00
 9.99999808e+00 9.99999978e+00 9.99999972e+00 9.99999978e+00
 9.99999968e+00 2.93903453e-01 9.99999944e+00 9.99999967e+00
 9.99999979e+00 9.99993200e+00 9.99999913e+00 4.89893097e+00
 9.99999963e+0

In [22]:
error = np.abs((xtest @ w + b[0] > 0) * 1 - (ytest > 0)).mean()

In [23]:
accuracy = 1 - error

In [24]:
print("Accuracy and  error on Test set is {} and {}".format(accuracy*100, error*100))

Accuracy and  error on Test set is 90.60000000000001 and 9.4


In [25]:
np.sum(alphas * ytrain)

9.237055564881302e-14

In [26]:
error1 = np.abs((xtrain @ w + b[0] > 0) * 1 - (ytrain > 0)).mean()
accuracy1 = 1 - error1
print("Accuracy and  error on Training set is {} and {}".format(accuracy1*100, error1*100))

Accuracy and  error on Training set is 93.45 and 6.550000000000001
